In [1]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib
from sklearn.feature_selection import mutual_info_classif, SelectKBest, f_classif

In [2]:
data = pd.read_csv(r"C:\Users\flyro\OneDrive\Documentos\data sets cd\Telco_Cusomer_Churn.csv")

In [3]:
data.head(4) #objetivo predecir el churn con un modelo de clasificacion posteriormente implementar una pipeline

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
data.describe()


,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [6]:
data.columns
def func(x):
    if x == "Yes":
        return 1
    else:
        return 0
    
data["churn_numerico"] = data["Churn"].apply(func)
data.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,churn_numerico
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,No,One year,No,Mailed check,56.95,1889.5,No,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No,0
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No,0
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No,0


In [27]:
X = data.drop(columns=["churn_numerico","Churn"])
y = data[["churn_numerico"]]

numeric_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

print(f"columnas categoricas {cat_cols}")
print(f"columnas numericas {numeric_cols}")
      

        

columnas categoricas ['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'TotalCharges']
columnas numericas ['SeniorCitizen', 'tenure', 'MonthlyCharges']


In [28]:
preprocessor = ColumnTransformer([("num", StandardScaler(), numeric_cols),("cat",OneHotEncoder(handle_unknown="ignore"), cat_cols)])

In [29]:

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, stratify = y, random_state=42)
scale_pos_weight = (y_train == 0).sum()/(y_train ==1).sum()
scale_pos_weight = scale_pos_weight.iloc[0]

In [30]:
xgb_model = XGBClassifier(n_estimators = 100,
                   max_depth = 3,
                   learning_rate=0.05,
                   subsample = 0.8,
                   colsample_bytree=0.8,
                   scale_pos_weight = scale_pos_weight,
                   random_state=42,
                   eval_metric = "logloss")

from sklearn.feature_selection import SelectFromModel

selector = SelectFromModel(xgb_model)

In [32]:
#from imblearn.pipeline import Pipeline as IPipeline
#from imblearn.over_sampling import SMOTE
pipeline = Pipeline([("preprocessor",preprocessor),("selector",selector),("modelo", xgb_model)])

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score, make_scorer
param_grid = {
    "modelo__max_depth": [3, 5, 7],
    "modelo__learning_rate": [0.01, 0.05, 0.1],
    "modelo__n_estimators": [100, 200, 300],
    "modelo__subsample": [0.7, 0.8, 1.0],
    "modelo__colsample_bytree": [0.7, 0.8, 1.0]
}

kappa_scorer = make_scorer(cohen_kappa_score)

grid_search = GridSearchCV(estimator=pipeline,          # tu pipeline
    param_grid=param_grid,       # la rejilla de hiperparámetros
    scoring=kappa_scorer,       # métrica que optimizas
    cv=3,                        # validación cruzada 3-fold
    n_jobs=-1,                   # usa todos los núcleos del procesador
    verbose=2
)

In [40]:
#entrenando el modelo 
grid_search.fit(X_train, y_train)
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)


Fitting 3 folds for each of 243 candidates, totalling 729 fits


C:\Users\flyro\anaconda3\lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [45]:
optimal_threshold = 0.4
y_pred_final = (y_pred_proba > optimal_threshold).astype(int)
print(f"exactitud {accuracy:.4f}")
print("classification", classification_report(y_test,y_pred))
print("confussion matrix", confusion_matrix(y_test,y_pred))
print("curva ROC", roc_auc_score(y_test,y_pred))

print("cohen kappa xgb", cohen_kappa_score(y_test,y_pred))

exactitud 0.7558
classification               precision    recall  f1-score   support

           0       0.89      0.77      0.82      1552
           1       0.53      0.73      0.61       561

    accuracy                           0.76      2113
   macro avg       0.71      0.75      0.72      2113
weighted avg       0.79      0.76      0.77      2113

confussion matrix [[1189  363]
 [ 153  408]]
curva ROC 0.7466904873477038
cohen kappa xgb 0.440711091413288


In [42]:
from sklearn.metrics import f1_score
best_pipeline = grid_search.best_estimator_
y_pred_proba = best_pipeline.predict_proba(X_test)[:,1]
for t in np.arange(0.3,0.7,0.05):
    y_pred_thresh = (y_pred_proba > t).astype(int)
    print(f"umbral {t:.2f} F1 clase1:", f1_score(y_test,y_pred_thresh,pos_label=1))

umbral 0.30 F1 clase1: 0.6098319850653391
umbral 0.35 F1 clase1: 0.6184895833333334
umbral 0.40 F1 clase1: 0.6204529855868223
umbral 0.45 F1 clase1: 0.6169296987087518
umbral 0.50 F1 clase1: 0.6126126126126126
umbral 0.55 F1 clase1: 0.6153846153846154
umbral 0.60 F1 clase1: 0.60580204778157
umbral 0.65 F1 clase1: 0.5891891891891892


In [46]:
def predecir_nuevos(datos_nuevos):
    return pipeline.predict(datos_nuevos)

joblib.dump(pipeline, "mi_modelo_entrenado.pkl")
print("modelo guardado cmo mi_modelo_entrenado.pkl")

#def cargar_y_predecir():
    
    #pipeline_cargada = joblib.load('mi_modelo_entrenado.pkl')
    
    # Cargar datos nuevos (ajusta la ruta)
    #nuevos = pd.read_csv(r"C:\Users\flyro\OneDrive\Documentos\nueva_data.csv")
    #predicciones = pipeline_cargada.predict(nuevos)
    
    #return predicciones

#print("🎉 ¡Plantilla ejecutada correctamente!")
    

modelo guardado cmo mi_modelo_entrenado.pkl


In [50]:
def cargar_y_predecir():
    
    pipeline_cargada = joblib.load('mi_modelo_entrenado.pkl')
    
    #Cargar datos nuevos (ajusta la ruta)
    nuevos = pd.read_csv(r"C:\Users\flyro\OneDrive\Documentos\nueva_data.csv")
    predicciones = pipeline_cargada.predict(nuevos)
    
    print("🔮 Predicciones generadas:")
    print(predicciones)
    print(f"\n📊 Total de predicciones: {len(predicciones)}")
    
    return predicciones

In [51]:
resultados = cargar_y_predecir()
resultados

NotFittedError: This ColumnTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
rf = RandomForestClassifier(n_estimators = 100,
                 max_depth = 3,
                 min_samples_split = 2,
                 min_samples_leaf = 1,
                 random_state = 42,
                 class_weight = "balanced")
#rf.fit(X_train,y_train)
#y_pred_rf = rf.predict(X_test)

#print("classification report", classification_report(y_test,y_pred_rf))
#print("confussion matrix", confusion_matrix(y_test,y_pred_rf))
#print("accuracy: ", accuracy_score(y_test,y_pred_rf))
#print("roc auc", roc_auc_score(y_test,y_pred_rf))


In [40]:
from sklearn.metrics import cohen_kappa_score
#kappa = cohen_kappa_score(y_test,y_pred_xgb)
#print("cohen kappa xgb", kappa)

In [5]:
"""cargando paqueterias necesarias"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score, cohen_kappa_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib
from sklearn.feature_selection import mutual_info_classif, SelectKBest, f_classif
"""carga de datos csv"""
data = pd.read_csv(r"C:\Users\flyro\OneDrive\Documentos\data sets cd\Telco_Cusomer_Churn.csv")
"""descripcion de la data"""
data.info()

data.describe()


data.columns
def func(x):
    if x == "Yes":
        return 1
    else:
        return 0
"""se define la funcion target y se aplica al data set para hacerlo binario"""    
data["churn_numerico"] = data["Churn"].apply(func)
data.head(10)


"""se separan los datos de entrada y de salida y se seleccionan las columnas"""
X = data.drop(columns=["churn_numerico","Churn"])
y = data["churn_numerico"]

"""se separan columnas categoricas de columnas numericas"""
numeric_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

print(f"columnas categoricas {cat_cols}")
print(f"columnas numericas {numeric_cols}")
 """se procesan las columnas categoricas y numericas, escalando las numericas y codificando las categoricas"""     
preprocessor = ColumnTransformer([("num", StandardScaler(), numeric_cols),("cat",OneHotEncoder(handle_unknown="ignore"), cat_cols)])

"""se separa el test de entrenamiento y prueba con un 30% para determinar cuanto se usara de prueba y el resto como entrenamiento"""
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, stratify = y, random_state=42)
"""se hace un balanceo de clases"""
scale_pos_weight = (y_train == 0).sum()/(y_train ==1).sum()
#scale_pos_weight = scale_pos_weight.iloc[0]
"""se define el modelo indicando los hiperparametros"""
xgb_model = XGBClassifier(n_estimators = 100,
                   max_depth = 3,
                   learning_rate=0.05,
                   subsample = 0.8,
                   colsample_bytree=0.8,
                   scale_pos_weight = scale_pos_weight,
                   random_state=42,
                   eval_metric = "logloss")
"""se define la pipeline con el modelo y el procesamiento de los datos seleccionando las mejores caracteristicas para el modelo"""
from sklearn.feature_selection import SelectFromModel

selector = SelectFromModel(xgb_model)

pipeline = Pipeline([("preprocessor",preprocessor),("selector",selector),("modelo", xgb_model)])

"""se importa gridsearchcv para encontrar la combinacion optima de hiperparametros"""
from sklearn.model_selection import GridSearchCV
"""se importan las metricas kappa score make scorer para evaluar el funcionamiento de este"""
from sklearn.metrics import cohen_kappa_score, make_scorer
param_grid = {
    "modelo__max_depth": [3, 5, 7],
    "modelo__learning_rate": [0.01, 0.05, 0.1],
    "modelo__n_estimators": [100, 200, 300],
    "modelo__subsample": [0.7, 0.8, 1.0],
    "modelo__colsample_bytree": [0.7, 0.8, 1.0]
}

kappa_scorer = make_scorer(cohen_kappa_score)

"""se incluye en gridsearch la pipeline y el modelo entrenado y con que metrica va a funcionar"""
grid_search = GridSearchCV(estimator=pipeline,          
    param_grid=param_grid,       # la rejilla de hiperparámetros
    scoring=kappa_scorer,       # métrica 
    cv=3,                        # validación cruzada 3-fold
    n_jobs=-1,                   # usa todos los núcleos del procesador
    verbose=2
)
"""se entrena el modelo con gridsearchcv y se toman los mejores estimadores"""
grid_search.fit(X_train, y_train)
best_pipeline = grid_search.best_estimator_
y_pred = best_pipeline.predict(X_test)
y_pred_proba = best_pipeline.predict_proba(X_test)[:,1]
accuracy = accuracy_score(y_test, y_pred)
"""se toma el mejor humbral para predecir la probabilidad y de ahi categorizar los mejores parametros"""
optimal_threshold = 0.4
y_pred_final = (y_pred_proba > optimal_threshold).astype(int)
print(f"mejores parametros {grid_search.best_params_}")
print(f"exactitud {accuracy:.4f}")
print("classification", classification_report(y_test,y_pred))
print("confussion matrix", confusion_matrix(y_test,y_pred))
print("curva ROC", roc_auc_score(y_test,y_pred))

print("cohen kappa xgb", cohen_kappa_score(y_test,y_pred))
"""se guarda el modelo entrenado para usarlo depues"""
joblib.dump(best_pipeline, "mi_modelo_entrenado.pkl")
print("modelo guardado cmo mi_modelo_entrenado.pkl")

"""se definen funciones para predecir nuevos resultados de nueva data"""
def predecir_nuevos(datos_nuevos):
    return best_pipeline.predict(datos_nuevos)


def cargar_y_predecir():
    
    pipeline_cargada = joblib.load('mi_modelo_entrenado.pkl')
    
    # Cargar datos nuevos
    nuevos = pd.read_csv(r"C:\Users\flyro\OneDrive\Documentos\nueva_data.csv")
    
    #  Convertir TotalCharges a numérico en los datos nuevos
    nuevos['TotalCharges'] = pd.to_numeric(nuevos['TotalCharges'], errors='coerce')
    nuevos['TotalCharges'] = nuevos['TotalCharges'].fillna(0)
    #  Y convertir a string para que coincida con el entrenamiento
    nuevos['TotalCharges'] = nuevos['TotalCharges'].astype(str)
    
    print(" Datos nuevos preparados:")
    print(f"   - Tipo de TotalCharges: {nuevos['TotalCharges'].dtype}")
    print(f"   - Primeros valores: {nuevos['TotalCharges'].head().tolist()}")
    
    predicciones = pipeline_cargada.predict(nuevos)
    
    print("🔮 Predicciones generadas:")
    print(predicciones)
    print(f"\n📊 Total de predicciones: {len(predicciones)}")
    print(f"📈 Clientes con Churn: {sum(predicciones)}")
    print(f"📉 Clientes sin Churn: {len(predicciones) - sum(predicciones)}")
    
    return predicciones




resultados = cargar_y_predecir()
resultados

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


C:\Users\flyro\anaconda3\lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


mejores parametros {'modelo__colsample_bytree': 0.8, 'modelo__learning_rate': 0.05, 'modelo__max_depth': 7, 'modelo__n_estimators': 100, 'modelo__subsample': 0.7}
exactitud 0.7558
classification               precision    recall  f1-score   support

           0       0.89      0.77      0.82      1552
           1       0.53      0.73      0.61       561

    accuracy                           0.76      2113
   macro avg       0.71      0.75      0.72      2113
weighted avg       0.79      0.76      0.77      2113

confussion matrix [[1189  363]
 [ 153  408]]
curva ROC 0.7466904873477038
cohen kappa xgb 0.440711091413288
modelo guardado cmo mi_modelo_entrenado.pkl
 Datos nuevos preparados:
   - Tipo de TotalCharges: object
   - Primeros valores: ['5450.7', '2962.0', '957.1', '857.25', '244.1']
🔮 Predicciones generadas:
[0 0 0 1 0 0 0 0 0 0 0 0 0 0]

📊 Total de predicciones: 14
📈 Clientes con Churn: 1
📉 Clientes sin Churn: 13


array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [7]:
from datetime import datetime

def predecir_y_guardar(ruta_datos_nuevos, ruta_guardado=None):
    """del modelo entrenado se carga con joblib.load"""
    try:
        print("cargando el modelo")
        pipeline = joblib.load("mi_modelo_entrenado.pkl")
        
        print("cargando datos nuevos")
        """se definen parametros en donde se carga el formato de csv o tipo excel"""
        if ruta_datos_nuevos.lower().endswith(".csv"):
            nuevos_datos = pd.read_csv(ruta_datos_nuevos)
        elif ruta_datos_nuevos.lower().endswith(".xlsx"):
            nuevos_datos = pd.read_excel(ruta_datos_nuevos)
        else:
            raise ValueError("Formato no soportado. usa .csv o .xlsx")
        
        nuevos_datos["TotalCharges"] = nuevos_datos["TotalCharges"].astype(str) 
        
        """con el modelo ya cargado y la data nueva ya cargada se hacen las predicciones y se toma en cuenta la probabilidad de exito"""
        print("hacer predicciones")
        predicciones = pipeline.predict(nuevos_datos)
        probabilidades = pipeline.predict_proba(nuevos_datos)[:,1]
        """con los nuevos datos se crea una copia
        -- se definen las variables de clasificación y la probabilidad de ser correcta la clasificación"""
        resultados = nuevos_datos.copy()
        resultados["prediccion_churn"] = predicciones
        resultados["probabilidad_churn"] = probabilidades
        """se categoriza de nuevo como 1 churn y 0 no churn.
        Se mide el riesgo con una función lambda con la probabilidad de churn"""
        resultados["estado_predicho"] = resultados["prediccion_churn"].map({1:"churn",0:"no churn"})
        resultados["riesgo"] = resultados["probabilidad_churn"].apply(lambda x:"alto" if x>0.7 else "medio" if x>0.4 else "bajo")
        
        """se crea la variable para saber cuando se corrio la predicción con el formato datetime y la versión del modelo
        con la que se hizo esta"""
        resultados["fecha_prediccion"] = datetime.now().strftime("%Y-%m-%d_%H:%M:%S" )
        resultados["modelo_version"] = "XGBoost_churn_v1"
        """si la ruta giradado es vacia se toma la fecha de al momento en el que se corrio y se gurada al final"""
        if ruta_guardado is None:
            timestamp = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
            ruta_guardado = f"predicciones_churn {timestamp}.csv"
            
        resultados.to_csv(ruta_guardado, index=False)
        """se imprimen los resultados de la prediccion se cuenta el numero de clientes
        los que se predijeron como 1 y los que se predijeron como 0"""
        print(f"✅ Listo! Se analizaron {len(resultados)} clientes")
        print(f"📈 Clientes que SÍ harán churn: {sum(predicciones)}")
        print(f"📉 Clientes que NO harán churn: {len(predicciones) - sum(predicciones)}")
        print(f"💾 Archivo guardado: {ruta_guardado}")
        
        return resultados
    
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

In [8]:
predecir_y_guardar(r"C:\Users\flyro\OneDrive\Documentos\data2.xlsx", r"C:\Users\flyro\OneDrive\Documentos\resultados2.csv")

cargando el modelo
cargando datos nuevos
hacer predicciones
✅ Listo! Se analizaron 11 clientes
📈 Clientes que SÍ harán churn: 4
📉 Clientes que NO harán churn: 7
💾 Archivo guardado: C:\Users\flyro\OneDrive\Documentos\resultados2.csv


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,prediccion_churn,probabilidad_churn,estado_predicho,riesgo,fecha_prediccion,modelo_version
0,7760-OYPDY,Female,0,No,No,2,Yes,No,Fiber optic,No,...,Yes,Electronic check,80.65,334.0,1,0.858062,churn,alto,2025-11-04_16:25:46,XGBoost_churn_v1
1,7639-LIAYI,Male,0,No,No,52,Yes,Yes,DSL,Yes,...,Yes,Credit card (automatic),223.00,223.5,0,0.302821,no churn,bajo,2025-11-04_16:25:46,XGBoost_churn_v1
2,2954-PIBKO,Female,0,Yes,Yes,69,Yes,Yes,DSL,Yes,...,Yes,Credit card (automatic),32.00,444.3,0,0.023501,no churn,bajo,2025-11-04_16:25:46,XGBoost_churn_v1
3,8012-SOUDQ,Female,1,No,No,43,Yes,Yes,Fiber optic,No,...,Yes,Electronic check,43.20,8765.4,1,0.671648,churn,medio,2025-11-04_16:25:46,XGBoost_churn_v1
4,9420-LOJKX,Female,0,No,No,15,Yes,No,Fiber optic,Yes,...,Yes,Credit card (automatic),21.30,342.5,0,0.458887,no churn,medio,2025-11-04_16:25:46,XGBoost_churn_v1
5,6575-SUVOI,Female,1,Yes,No,25,Yes,Yes,DSL,Yes,...,Yes,Credit card (automatic),22.30,66543.21,0,0.326278,no churn,bajo,2025-11-04_16:25:46,XGBoost_churn_v1
6,7495-OOKFY,Female,1,Yes,No,8,Yes,Yes,Fiber optic,No,...,Yes,Credit card (automatic),80.65,2213.4,1,0.870369,churn,alto,2025-11-04_16:25:46,XGBoost_churn_v1
7,4667-QONEA,Female,1,Yes,Yes,60,Yes,No,DSL,Yes,...,Yes,Credit card (automatic),74.85,44356.32,0,0.150257,no churn,bajo,2025-11-04_16:25:46,XGBoost_churn_v1
8,1658-BYGOY,Male,1,No,No,18,Yes,Yes,Fiber optic,No,...,Yes,Electronic check,95.45,665478.0,1,0.869618,churn,alto,2025-11-04_16:25:46,XGBoost_churn_v1
9,8769-KKTPH,Female,0,Yes,Yes,63,Yes,Yes,Fiber optic,Yes,...,Yes,Credit card (automatic),99.65,443.65,0,0.248001,no churn,bajo,2025-11-04_16:25:46,XGBoost_churn_v1


In [4]:
import os
print("¿Existe el modelo?", os.path.exists("mi_modelo_entrenado.pkl"))

¿Existe el modelo? True


In [2]:
import os
os.getcwd()


'C:\\Users\\flyro'